In [6]:
from pathlib import Path

In [7]:
test_folder = Path('./test')

In [8]:
[x for x in test_folder.iterdir() if test_folder.is_dir()]

[WindowsPath('test/frieren_clip1.mp4')]

In [9]:
path = Path('./test')

# path.iterdir()
[x for x in test_folder.iterdir() if test_folder.is_dir()]

[WindowsPath('test/frieren_clip1.mp4')]

In [10]:
import shutil
dst = Path("../frontend/src/assets")

for file in test_folder.iterdir():
    if file.is_file():
        shutil.move(file, dst / file.name)

In [11]:
def move_files(src: Path, dst: Path):
    for file in src.iterdir():
        if file.is_file():
            shutil.move(file, dst / file.name)

In [12]:
move_files(Path("./test"), Path("../frontend/src/assets"))

## Testing out scenedetect

In [13]:
from scenedetect import *
print(f"Finding path..")
path = "./test/frieren_clip1.mp4"

print(f"Finding cuts...")
scene_list = detect(path, 
                    AdaptiveDetector(
                        adaptive_threshold=4.0,
                        min_scene_len=25, 
                        min_content_val=35.0 
                    )
                )

print(f"SCENE LIST:\n {scene_list}")

print(f"Splitting scenes...")
split_video_ffmpeg(path, scene_list, output_dir="./output_test")

Finding path..
Finding cuts...


OSError: Video file not found.

In [1]:
from scenedetect import SceneManager, open_video
from scenedetect.detectors import ContentDetector, AdaptiveDetector
from scenedetect.scene_manager import Interpolation

path = "./test/frieren_clip1.mp4"

video = open_video(path)

scene_manager = SceneManager()    
scene_manager.interpolation = Interpolation.AREA    

scene_manager.clear_detectors()

"""ContentDetector Settings"""
# scene_manager.add_detector(
#     ContentDetector(
#         threshold=32.0,
#         min_scene_len=30
#     )
# )

"""AdaptiveDetector Settings"""
scene_manager.add_detector(
    AdaptiveDetector(
        adaptive_threshold=3.5,
        min_scene_len=10,
        min_content_val=30.0
    )
)
scene_manager.downscale = 3


scene_manager.detect_scenes(frame_source=video)
scene_list = scene_manager.get_scene_list()

print(f"Splitting scenes..")
print(scene_list)
split_video_ffmpeg(path, scene_list, output_dir="./output_test")

c:\Users\amiri\OneDrive\Documents\AMVerge\backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downscale factor will be ignored because auto_downscale=True!


Splitting scenes..
[(00:00:00.000 [frame=0, fps=23.976], 00:00:01.627 [frame=39, fps=23.976]), (00:00:01.627 [frame=39, fps=23.976], 00:00:03.378 [frame=81, fps=23.976]), (00:00:03.378 [frame=81, fps=23.976], 00:00:08.133 [frame=195, fps=23.976]), (00:00:08.133 [frame=195, fps=23.976], 00:00:12.262 [frame=294, fps=23.976]), (00:00:12.262 [frame=294, fps=23.976], 00:00:13.180 [frame=316, fps=23.976]), (00:00:13.180 [frame=316, fps=23.976], 00:00:15.432 [frame=370, fps=23.976]), (00:00:15.432 [frame=370, fps=23.976], 00:00:19.186 [frame=460, fps=23.976]), (00:00:19.186 [frame=460, fps=23.976], 00:00:20.938 [frame=502, fps=23.976]), (00:00:20.938 [frame=502, fps=23.976], 00:00:27.694 [frame=664, fps=23.976]), (00:00:27.694 [frame=664, fps=23.976], 00:00:32.699 [frame=784, fps=23.976]), (00:00:32.699 [frame=784, fps=23.976], 00:00:34.826 [frame=835, fps=23.976]), (00:00:34.826 [frame=835, fps=23.976], 00:00:43.585 [frame=1045, fps=23.976]), (00:00:43.585 [frame=1045, fps=23.976], 00:00:48.

NameError: name 'split_video_ffmpeg' is not defined

### Keyframe System Design

1) Generate keyframes from the video using FFMPEG and subprocess
2) Use those keyframes to trim the video around each keyframe, 1 second around
3) Use scenedetect on the updated video

In [2]:
import subprocess
import json

def generate_keyframes(video_path: str):
    """Generates keyframe for a given video"""
    cmd = [
        "ffprobe",                      # Launches the ffprobe executable
        "-skip_frame", "nokey",         # ONLY look at keyframes, skip others (speed)
        "-select_streams", "v:0",       # Only looks at the first video stream
        "-show_frames",                 # Shows metadata for EVERY frame
        "-show_entries", "frame=best_effort_timestamp_time",  # Filter, we only get the frame of the data
        "-of", "json",                  # Export as json format
        video_path
    ]

    print(f"Execting command..")
    # Executing system command
    result = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,  # Return strings instead of bytes
        check=True  # Crash if the command fails
    )

    data = json.loads(result.stdout)
    
    return [
        float(frame["best_effort_timestamp_time"])
        for frame in data.get("frames", [])
        if "best_effort_timestamp_time" in frame
    ]

In [3]:
def keyframe_windows(keyframes, radius=1.0):
    """Generates keyframe windows for """
    windows = [(max(0, key - radius), key + radius) for key in keyframes]

    windows.sort()
    merged = [windows[0]]  
    
    # Ensures that none of the windows overlap each other
    for start, end in windows[1:]:
        last_start, last_end = merged[-1]
        if start <= last_end:
            merged[-1] = (last_start, max(last_end, end)) # Replace appended start with the last end
        else:
            merged.append((start, end))
    
    return merged

In [4]:
import os
import subprocess

def trim_keyframes(video_path: str, output_dir="./keyframe_clips", radius=1.0):
    os.makedirs(output_dir, exist_ok=True)

    # generating keyframes
    print(f"Generating keyframes..")
    keyframes = generate_keyframes(video_path)
    if not keyframes:
        return []
    print(f"Keyframes: {keyframes}")

    # generating keyframe windows
    print(f"Generating keyframe windows...")
    windows = keyframe_windows(keyframes, radius)
    print(f"Windows: {windows}")
    clips = []
    
    print(f"Trimming video from keyframes..")
    # going through each keyframe window to trim the video
    for i, (start, end) in enumerate(windows):
        out_path = os.path.join(output_dir, f"kf_clip_{i:04d}.mp4")

        cmd = [
            "ffmpeg",
            "-y",
            "-ss", str(start),  # trim from start of keyframe
            "-to", str(end),    # trim until end of keyframe 
            "-i", video_path,
            "-c", "copy",
            out_path
        ]

        subprocess.run(
            cmd,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            check=True
        )

        clips.append(
            {
                "clip_path": out_path,
                "window_start": start,
                "window_end": end
            }
        )
    return clips

In [5]:
from scenedetect import SceneManager, open_video
from scenedetect.detectors import AdaptiveDetector, ContentDetector

def detect_and_trim_scenes(
        original_video_path: str,
        keyframe_clips: list,
        output_dir="./output_test"
):
    os.makedirs(output_dir, exist_ok=True)

    final_scenes = []
    scene_idx = 0

    for clip in keyframe_clips:
        clip_path = clip["clip_path"]
        offset = clip["window_start"]

        video = open_video(clip_path) 
        manager = SceneManager()
        manager.add_detector(
            ContentDetector(
                    threshold=25.0, 
            )
        )
        manager.detect_scenes(video) # detecting all the keyframes here
        
        scenes = manager.get_scene_list()
        print(f"SCENES: {scenes}")

        for scene in scenes:
            scene_start = scene[0].get_seconds()
            scene_end = scene[1].get_seconds()

            # getting the actual timestamp
            abs_start = offset + scene_start
            abs_end = offset + scene_end
            
            out_path = os.path.join(output_dir, f"scene_{scene_idx:04d}.mp4")

            cmd = [
                "ffmpeg", "-y",
                "-ss", str(abs_start),
                "-to", str(abs_end),
                "-i", original_video_path,
                "-c", "copy",
                out_path
            ]

            subprocess.run(cmd, 
                           stdout=subprocess.DEVNULL,
                           stderr=subprocess.DEVNULL,
                           check=True)
            
            final_scenes.append({
                "scene_index": scene_idx,
                "start": abs_start,
                "end": abs_end,
                "path": out_path
            })

            scene_idx += 1
    return final_scenes

In [ ]:
import shutil
import time

video_path = "./test/frieren_clip1.mp4"

trimmed_keyframes = trim_keyframes(video_path, radius=1.0)

print(f"Using trimmed clips for scene detection...")
detect_and_trim_scenes(original_video_path=video_path,
                       keyframe_clips=trimmed_keyframes,
                       output_dir="./output_test")
time.sleep(0.2)
shutil.rmtree("./keyframe_clips", ignore_errors=True)

##### FIX SCENES NOT CUTTING, 

Generating keyframes..
Execting command..
Keyframes: [0.0, 1.626625, 3.378375, 8.133125, 12.26225, 13.930583, 15.432083, 19.185833, 20.937583, 27.694333, 32.699333, 34.826458, 43.585208, 48.089708, 51.468083, 59.726333, 63.730333, 70.236833, 77.744333, 84.375958, 87.504083]
Generating keyframe windows...
Windows: [(0, 4.378375), (7.133125, 9.133125), (11.26225, 16.432083), (18.185833, 21.937583), (26.694333, 28.694333), (31.699333000000003, 33.699333), (33.826458, 35.826458), (42.585208, 44.585208), (47.089708, 49.089708), (50.468083, 52.468083), (58.726333, 60.726333), (62.730333, 64.730333), (69.236833, 71.236833), (76.744333, 78.744333), (83.375958, 85.375958), (86.504083, 88.504083)]
Trimming video from keyframes..
Using trimmed clips for scene detection...
SCENES: [(00:00:00.000 [frame=0, fps=23.976], 00:00:01.627 [frame=39, fps=23.976]), (00:00:01.627 [frame=39, fps=23.976], 00:00:03.378 [frame=81, fps=23.976]), (00:00:03.378 [frame=81, fps=23.976], 00:00:04.463 [frame=107, fps=2